In [1]:
import pandas as pd

diabetic = pd.read_csv("New_train_set.csv")
diabetic = diabetic.drop(diabetic.columns[0],axis = 1)

In [11]:
from sklearn.model_selection import KFold
import numpy as np
from collections import Counter

from imblearn.under_sampling import RandomUnderSampler
from sklearn.metrics import f1_score

def back_wrapper(clf,X_raw,y,k,n_bag):
    X = X_raw.copy()
    Wrap_accuracy_old = 0
    Wrap_accuracy_new = 0.00001
    n = X.shape[1]
    #generate an empty dataframe to save features
    Wrap_filt = X.copy()
    
    kf = KFold(n_splits=k, random_state=42, shuffle=False)
    rus = RandomUnderSampler(replacement=True)
    
    while(Wrap_accuracy_old <  Wrap_accuracy_new):
        Wrap_accuracy_old = Wrap_accuracy_new
        avg_accuracies_list = []

        for i in range(Wrap_filt.shape[1]):
            accur_list = []
            for train_index, test_index in kf.split(Wrap_filt):                
                y_Pred = []
                for j in range(n_bag):
                    # sample N balanced bag to do N times of prediction
                    X_res, y_res = rus.fit_resample(Wrap_filt.drop(Wrap_filt.columns[i],axis=1).iloc[train_index,:], y.iloc[train_index])
                    clf.fit(X_res, y_res)
                    y_pred = clf.predict(Wrap_filt.drop(Wrap_filt.columns[i],axis=1).iloc[test_index,:])
                    y_Pred.append(y_pred)
                # turning list of prediction in to ndarray
                y_Pred = np.array(y_Pred)
                # majority vote using N prediction
                Y_Pred=[]
                for j in range(y_Pred.shape[1]):
                    Y_Pred.append((Counter(y_Pred[:,j]).most_common())[0][0])
                Y_Pred = np.array(Y_Pred)
                
                score = f1_score(y.iloc[test_index],y_pred,average = "weighted")
                accur_list.append(score)
            avg_accuracies_list.append(np.mean(accur_list))
            
        # renew the accuracy
        Wrap_accuracy_new = np.sort(avg_accuracies_list)[-1]                 
        print(n,"selection is done")
        
        if Wrap_accuracy_old <  Wrap_accuracy_new:
            # add the highest accuracy feature to the selected dataframe
            Wrap_filt = Wrap_filt.drop(Wrap_filt.columns[np.argsort(avg_accuracies_list)[-1]],axis=1)          
            n = n - 1            
        else:
            break

    return Wrap_accuracy_old,n,Wrap_filt

In [3]:
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import Perceptron
from sklearn.svm import LinearSVC
from sklearn.neighbors import KNeighborsClassifier

clf1 = GaussianNB()
clf2 = Perceptron(random_state=42, n_jobs=-1)
clf3 = RandomForestClassifier(n_estimators='warn',n_jobs=-1)
clf4 = LinearSVC()
clf5 = KNeighborsClassifier(n_neighbors=5)
clf6 = LogisticRegression()

In [ ]:
logit_acc, logit_num_feature, logit_features = back_wrapper(clf6,diabetic.iloc[:,:-1],diabetic.iloc[:,-1],5,15)

In [ ]:
logit_features.to_csv("Logit_Backward_Bagging.csv",index=False)
logit_acc

In [ ]:
NB_acc, NB_num_feature, NB_features = back_wrapper(clf1,diabetic.iloc[:,:-1],diabetic.iloc[:,-1],5,15)

In [ ]:
NB_features.to_csv("NB_Backward_Bagging.csv",index=False)
NB_acc

In [ ]:
Per_acc, Per_num_feature, Per_features = back_wrapper(clf2,diabetic.iloc[:,:-1],diabetic.iloc[:,-1],5,15)

In [ ]:
Per_features.to_csv("Per_Backward_Bagging.csv",index=False)
Per_acc

In [ ]:
RF_acc, RF_num_feature, RF_features = back_wrapper(clf3,diabetic.iloc[:,:-1],diabetic.iloc[:,-1],5,15)

In [ ]:
RF_features.to_csv("RF_Backward_Bagging.csv",index=False)
RF_acc

In [ ]:
SVM_acc, SVM_num_feature, SVM_features = back_wrapper(clf4,diabetic.iloc[:,:-1],diabetic.iloc[:,-1],5,15)

In [ ]:
SVM_features.to_csv("SVM_Backward_Bagging.csv",index=False)
SVM_acc

In [ ]:
knn_acc, knn_num_feature, knn_features = back_wrapper(clf5,diabetic.iloc[:,:-1],diabetic.iloc[:,-1],5,15)

In [ ]:
knn_features.to_csv("knn_Backward_Bagging.csv",index=False)
knn_acc

In [ ]:
modelname = ['NB',"Perceptron","RF","SVM","KNN","Logit"]
summary_acc = pd.DataFrame({"accuracy":[NB_acc,Per_acc,RF_acc,SVM_acc,knn_acc,logit_acc]},index=modelname)
summary_acc.to_csv("Accuracy_Backward_Bagging.csv")

In [12]:
back_wrapper(clf3,diabetic.iloc[:,:-1],diabetic.iloc[:,-1],5,15)

87 selection is done
86 selection is done


(0.6968215048152891,
 86,
        time_in_hospital  num_lab_procedures  num_procedures  num_medications  \
 0              0.000000            0.106870        0.000000           0.0875   
 1              0.076923            0.000000        0.500000           0.2000   
 2              0.076923            0.221374        0.000000           0.1750   
 3              0.153846            0.190840        0.166667           0.0875   
 4              0.307692            0.503817        0.166667           0.1250   
 5              0.230769            0.450382        0.000000           0.0625   
 6              0.153846            0.450382        0.000000           0.2250   
 7              0.461538            0.427481        0.000000           0.1750   
 8              0.076923            0.007634        0.000000           0.2125   
 9              0.076923            0.442748        0.000000           0.0625   
 10             0.076923            0.030534        1.000000           0.0750   
 1